# explore_sentiment.py: Python script extracted from notebooks/explore_sentiment.ipynb
# Purpose: Test sentiment analysis and keyword extraction for Task 2


In [1]:
import pandas as pd
import spacy
from transformers import pipeline
from collections import Counter

ModuleNotFoundError: No module named 'spacy'

# Load cleaned reviews

In [ ]:
df = pd.read_csv('data/processed/reviews_clean.csv')
print(df.shape)
print(df.head())

# Initialize DistilBERT sentiment pipeline

In [ ]:
sentiment_analyzer = pipeline('sentiment-analysis', 
model='distilbert-base-uncased-finetuned-sst-2-english')


# Sample 50 reviews

In [ ]:
sample_df = df.sample(50, random_state=42)

# Apply sentiment analysis

In [ ]:
def get_sentiment(text):
    result = sentiment_analyzer(text[:512])[0]  # Truncate to 512 tokens
    return result['label'], result['score']

sample_df['sentiment'] = sample_df['review'].apply(lambda x: get_sentiment(x)[0])
sample_df['sentiment_score'] = sample_df['review'].apply(lambda x: get_sentiment(x)[1])
print(sample_df[['review', 'sentiment', 'sentiment_score']].head())    

# Aggregate by bank and rating

In [ ]:
sentiment_by_bank = sample_df.groupby(['bank', 'rating'])['sentiment_score'].mean().unstack()
print(sentiment_by_bank)


# Load spaCy model

In [ ]:
nlp = spacy.load('en_core_web_sm')


# Function to extract keywords

In [ ]:
def extract_keywords(text):
    doc = nlp(text.lower())
    keywords = [token.text for token in doc if token.pos_ in ['NOUN', 'PROPN'] or token.dep_ == 'compound']
    return keywords

# Apply to sample

In [ ]:
sample_df['keywords'] = sample_df['review'].apply(extract_keywords)
print(sample_df[['review', 'keywords']].head())

# Group keywords by bank

In [ ]:
keywords_by_bank = sample_df.groupby('bank')['keywords'].apply(lambda x: Counter([kw for sublist in x for kw in sublist]))
for bank, counter in keywords_by_bank.items():
    print(f"{bank}: {counter.most_common(10)}")


# Example manual theme clustering

In [ ]:
themes = {
    'Account Access Issues': ['login', 'password', 'authentication'],
    'Transaction Performance': ['transfer', 'payment', 'slow'],
    'User Interface': ['ui', 'design', 'navigation']
}
print('Sample Themes:', themes)